# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7f5d30572150>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
5721,0.8,914.50,-200,4.192061,721.25,-200.0,890.50,-200.0,1314.75,816.00,16.825,72.875000,1.385085
4770,1.4,856.75,-200,6.015120,812.50,51.0,1040.25,14.0,1183.25,622.25,19.325,34.750000,0.771047
4932,-200.0,1066.00,-200,6.335782,827.25,-200.0,751.00,-200.0,1479.25,1024.25,20.100,65.850002,1.532109
6776,1.4,1086.00,-200,5.101134,768.50,190.0,821.75,100.0,1196.00,856.75,12.375,73.775000,1.057370
8244,0.5,893.00,-200,1.076022,506.50,59.4,1110.50,51.4,794.25,488.50,6.900,42.225000,0.422400


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [5]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]


In [6]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [7]:
np.random.seed(61506)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data


The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [8]:

def cal_non_missing_mean(dataframe):
    """ 
        Define a function to calculate mean value of the non-missing values for each feature and return the 
        the result as an numpy array.
    """  
    
    col_names = list(dataframe.columns) # obtain the columns' name from dataframe
    ncols = len(col_names) # obtain the columns' number from dataframe
    mean_non_missing = np.zeros(ncols - 1) # a numpy array to store mean values of non-missing for each feature except
                                           # target value
    
    for i in range(1, ncols):
        mean_non_missing[i - 1] = (dataframe[col_names[i]][dataframe[col_names[i]] != -200]).mean()
    
    return mean_non_missing

mean_non_missing = cal_non_missing_mean(data_training_unproc) # calculate mean values of the non-missing values for
                                                              # each feature by defined function "cal_non_missing_mean"

def impute(dataframe, mean_non_missing):
    """
        Define a function to fill the missing values in each feature with the mean values calculated by non-missing 
        values in corresponding features
        
        'dataframe' is the dataframe needed to be filled.
    """
    
    col_names = list(dataframe.columns) # obtain columns's name
    ncols = len(col_names) # obtain columns' number 
    
    for i in range(1, ncols):
        dataframe[col_names[i]][dataframe[col_names[i]] == -200] = mean_non_missing[i - 1]
    
    return dataframe

data_training_imputed = impute(data_training_unproc.copy(), mean_non_missing) # impute training set by function "impute"


# calculate mean and standard deviation for each feature in training set and store them.
def cal_mean_for_standardize(dataframe):
    mean_val =  np.array(dataframe.mean())
    
    return mean_val

def cal_std_for_standardize(dataframe):
    std_val = np.array(dataframe.std())
    
    return std_val
mean_for_standardize = cal_mean_for_standardize(data_training_imputed)
std_for_standardize = cal_std_for_standardize(data_training_imputed)


# Define a function to standardize data set.
def standardize(dataframe, mean, std):
    """
        Define a function to standardize data set by mean and standard deviation values of each feature in training
        set
    """
    col_names = list(dataframe.columns)
    ncols = len(col_names)
    
    for i in range(1, ncols):
        dataframe[col_names[i]] = (dataframe[col_names[i]] - mean[i])/std[i]
    
    return dataframe

# standardize the imputed training set by define function 'standardize()'
data_training_standardized = standardize(data_training_imputed.copy(), mean_for_standardize, std_for_standardize)
print(data_training_standardized)

      CO(GT)  PT08.S1(CO)      C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  \
1649     1.9     0.127188  9.866060e-02   2.565668e-01 -0.558031   
359      2.0    -0.044581 -1.202716e-01   2.869661e-02 -0.654674   
4392     5.4     1.475457  3.046813e+00   2.543996e+00  1.814547   
4058     1.7     0.000000  2.456276e-16  -4.409505e-16 -0.712660   
8080     4.3     0.000000  2.456276e-16  -4.409505e-16  1.683113   
...      ...          ...           ...            ...       ...   
8932     2.2     0.463667 -1.676381e-01  -2.269539e-02  0.373604   
4831     0.8    -1.456380 -9.467532e-01  -1.042778e+00 -0.727156   
9071     2.5     0.602494  2.810881e-01   4.359540e-01  0.845704   
1983     2.3     0.413077  6.807853e-01   8.024857e-01 -0.137636   
6405     3.9     1.383690  1.203757e+00   1.239803e+00  1.688912   

      PT08.S3(NOx)   NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)         T        RH  \
1649  1.255511e-01 -0.257835      0.463279     0.177278  1.116524 -1.534987   
359   5.926525e-01 -0.579

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [9]:
# target variable is "CO(GT)"
# organize the target values for dataframe to a numpy array vector
def output_target_vector(target_df):
    target_vector = np.array(target_df["CO(GT)"]).reshape(-1, 1)
    
    return target_vector

yTrain = output_target_vector(data_training_standardized.copy())


# organize the feature values from dataframe to a numpy array vector
def output_feature_matrix(target_df):
    feature_matrix = np.hstack((np.ones(target_df.shape[0]).reshape(-1, 1), 
                                np.array(target_df.drop(["CO(GT)"], axis = 1))))
    
    return feature_matrix
    
XTrain = output_feature_matrix(data_training_standardized.copy())

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [10]:
# preprocess the missing values in validation data and standardize the validation data:

#     impute validation data set through predefined fucntion 'impute()' and the mean values of non-missing values 
# computed with training data                                                                    
data_val_imputed = impute(data_val_unproc.copy(), mean_non_missing)


#     standardize validation data set by predefined function 'standardize()' and the means and standards deviations
# computed from the training data.
data_val_standardized = standardize(data_val_imputed.copy(), mean_for_standardize, std_for_standardize) 

#     organize validation data set to target vector named yVal and a design matrix named XVal, these will be used
# in validation process.
yVal = output_target_vector(data_val_standardized.copy())
XVal = output_feature_matrix(data_val_standardized.copy())


####################################################################################################################
# create grid for hyperparameters
# define a function to create grid for mini-batch size S
def get_S(start, end, num):
    grid_S = np.linspace(start, end, num).astype("int") # mini-batch size should be an integer
    
    return grid_S

# define functions to create grids for learning rate 'eta' and regularization parameter 'gamma'
def get_eta(start, end, num):
    grid_eta = np.logspace(start, end, num)
    
    return grid_eta

def get_gamma(start, end, num):
    grid_gamma = np.logspace(start, end, num)
    
    return grid_gamma

# initialize five values for each hyperparameters to be checked
check_grid_S = get_S(1, 20, 5)
check_grid_eta = get_eta(-2, 1, 5)
check_grid_gamma = get_eta(-2, 1, 5)


# define a function to fit the training data and update weight with specified hyperparameter
def fit(train_input, train_target, eta, gamma, S):
    """
        A function to update weights by mini-batch gradient descent, returns the updated weights.
        'eta' is learning rate, 'gamma' is regularization parameter, and 'S' is mini-batch size
    """
    
    # initialization
    weight = np.random.randn(train_input.shape[1]).reshape(-1, 1) # initialize a random weight
    example_num = train_input.shape[0] # get the examples' number in training data set
    index = [i for i in range(example_num)]
    stop = 200 # iteration for gradient descent stop after 200 iterations
    
    for i in range(stop):
        mini_batch_index = np.random.choice(index, S, replace = False)
        
        # construct design matrix and output vector for minibatch
        mini_X = train_input[mini_batch_index[0], :].reshape(1, -1)
        mini_y = train_target[mini_batch_index[0], :].reshape(1, -1)
        for s in range(1, S):
            mini_X = np.vstack((mini_X, train_input[mini_batch_index[s], :]))
            mini_y = np.vstack((mini_y, train_target[mini_batch_index[s], :]))
        
        # update weight by gradient descent
        weight = weight - eta*((-2/S * mini_X.T @ mini_y) + (2/S * mini_X.T @ mini_X @ weight) + gamma * weight)
    
    return weight


# define a function to calculate RMSE according to updated weight
def RMSE(weight, test_input, test_target):
    """
        a function to calculate RMSE
        test_input could be training examples in validation set or test set;
        test_output could be desired value in validation set or test set
    """
    
    return np.sqrt(((test_input @ weight - test_target)**2).sum()/test_target.shape[0])

# define a validation function to choose the best hyperparameters, return the best hyperparameters to be stored.
def validation(param_set, train_input, train_target, val_input, val_target):
    """
        param_set is a list with 3 tuple elements, each element contains the arugments to create a grid for learning
        rate, mini-batch size, and regularization parameter.
    """
    grid_S = get_S(param_set[0][0], param_set[0][1], param_set[0][2])
    grid_eta = get_eta(param_set[1][0], param_set[1][1], param_set[1][2])
    grid_gamma = get_gamma(param_set[2][0], param_set[2][1], param_set[2][2])
    
    
    # store each combination of hyperparameters as a tuple in a list
    com_hyper = []
    
    # store the errors when using corresponding hyperparameters' combinationsin a list
    error_list = []
    
    # Compute the RMSE values for models with different combination of eta, gamma, S and choose the combination which
    # could lead the model's RMSE to minimum
    for num_S in range(param_set[0][2]):
        for num_eta in range(param_set[1][2]):
            for num_gamma in range(param_set[2][2]):
                eta = grid_eta[num_eta]
                gamma = grid_gamma[num_gamma]
                S = grid_S[num_S]
                val_weight = fit(train_input, train_target, eta, gamma, S) # fit the model using current 
                                                                           # hyperparameters
                error = RMSE(val_weight, val_input, val_target) # compute the RMSE of current model
                com_hyper.append((eta, gamma, S)) # store current combination of eta, gamma, S
                error_list.append(error) # store current RMSE
    
    return com_hyper[np.argmin(error_list)]

# choose the best mini-batch size, learning rate, and regularization parameter and save them
opt_eta, opt_gamma, opt_S = validation([(10, 1000, 5), (-2, 0, 5), (-2, 0, 10)],
                                      XTrain, yTrain,
                                      val_input = XVal, val_target = yVal)

print("best learning rate eta is : {}".format(opt_eta))
print("best regularization hyperparameter gamma is : {}".format(opt_gamma))
print("best mini_batch_size S is : {}".format(opt_S))
print("lowest RMSE for validation set is : {}".format(RMSE(fit(XTrain, yTrain, opt_eta, opt_gamma, opt_S), 
                                                           XVal, yVal)))
                


/home/songyangli/anaconda3/envs/NewBase/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: overflow encountered in square


best learning rate eta is : 0.1
best regularization hyperparameter gamma is : 0.027825594022071243
best mini_batch_size S is : 1000
lowest RMSE for validation set is : 0.441684533425697


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [11]:
# put original training dataframe and validation dataframe together to a new training unprocessed dataframe
new_train_unproc = pd.concat([data_training_unproc.copy(), data_val_unproc.copy()])

# preprocess new training data set
# impute the missing value in new training data set
new_mean_for_impute = cal_non_missing_mean(new_train_unproc.copy())
new_train_imputed = impute(new_train_unproc.copy(), new_mean_for_impute)

# standardize the new imputed training data set
new_mean_for_stand = cal_mean_for_standardize(new_train_imputed.copy())
new_std_for_stand = cal_std_for_standardize(new_train_imputed.copy())
new_train_standardized = standardize(new_train_imputed.copy(), new_mean_for_stand, new_std_for_stand)

# organize new training data set to target vector and feature design matrix
new_XTrain = output_feature_matrix(new_train_standardized.copy())
new_yTrain = output_target_vector(new_train_standardized.copy())


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [12]:
# preprocess the test data
# impute missing value and standardize
data_test_imputed = impute(data_test_unproc.copy(), new_mean_for_impute)
data_test_standardized = standardize(data_test_imputed.copy(), new_mean_for_stand, new_std_for_stand)

# organize features and target values to numpy vector and matrix
new_XTest = output_feature_matrix(data_test_standardized.copy())
new_yTest = output_target_vector(data_test_standardized.copy())



# train a new model and get new updated weight
new_updated_weight = fit(new_XTrain, new_yTrain, opt_eta, opt_gamma, opt_S)



# calculate the RMSE
error = RMSE(new_updated_weight, new_XTest, new_yTest)
#report the RMSE
print("RMSE for testing dataset is : {}".format(error))


RMSE for testing dataset is : 0.5083467429694629


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [14]:
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# First, I will separate the target values from feature values because the target values neither need to be imputed,
# nor need to be standardized
# Here I don't need a pipeline or function for separation because this operation only occurs one time.
data_training_attribute_unproc = data_training_unproc.drop(["CO(GT)"], axis = 1)
data_training_target = data_training_unproc["CO(GT)"]
data_val_attribute_unproc = data_val_unproc.drop(["CO(GT)"], axis = 1)
data_val_target = data_val_unproc["CO(GT)"]
data_test_attribute_unproc = data_test_unproc.drop(["CO(GT)"], axis = 1)
data_test_target = data_test_unproc["CO(GT)"]


# define an instance of simpleimputer and define an instance of standardscaler
simpleimp = SimpleImputer(missing_values = -200, strategy = "mean")
standardsca = StandardScaler()

# define a instance of Pipeline named "preprocess_1" to preprocess the data 
preprocess_1 = Pipeline([("imputer", simpleimp),
                      ("scaler", standardsca)])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [15]:
# use the Pipeline "preprocess_1" to impute and standardize the original training data and validation data
XTrain_rf = preprocess_1.fit_transform(data_training_attribute_unproc.copy())
yTrain_rf = np.array(data_training_target.copy()).reshape(-1, 1)

XVal_rf = preprocess_1.transform(data_val_attribute_unproc.copy())
yVal_rf = np.array(data_val_target.copy()).reshape(-1, 1)

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [16]:
from sklearn.model_selection import PredefinedSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

val_fold = np.zeros((XTrain_rf.shape[0] + XVal_rf.shape[0], 1))
val_fold[0:XTrain_rf.shape[0], 0] = -1
pre_split = PredefinedSplit(val_fold)

# stack the training set and validation set together
whole_X_rf = np.vstack((XTrain_rf, XVal_rf))
whole_y_rf = np.vstack((yTrain_rf, yVal_rf)) 


# create the values for n_estimators, max_features, and max_samples separately.
n_estimators_set = np.linspace(10, 200, 7).astype("int")
max_features_set = np.linspace(1, whole_X_rf.shape[1], 7).astype("int")
max_samples_set = np.linspace(0.2*XTrain_rf.shape[0], XTrain_rf.shape[0], 7).astype("int")


# create the grid for all parameters
param_grid = dict(n_estimators = n_estimators_set,
                 max_features = max_features_set,
                 max_samples = max_samples_set)


# train the random forest regressor by using GridSearchCv to find the best parameters
grid_regressor = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, 
                             cv = pre_split, scoring = "neg_mean_squared_error")
grid_regressor.fit(whole_X_rf, whole_y_rf.reshape(1, -1)[0])


# print best parameters
print(grid_regressor.best_params_)


{'max_features': 4, 'max_samples': 5372, 'n_estimators': 105}


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [17]:
# concatenate the unprocessed training data and unprocessed validation data
whole_training_attribute_unproc = pd.concat([data_training_attribute_unproc.copy(), data_val_attribute_unproc.copy()],
                                           ignore_index = True)
whole_training_target = pd.concat([data_training_target.copy(), data_val_target.copy()],
                                 ignore_index = True)


# We create a new pipeline named "preprocess_whole" for the whole_training dataset and test dataset
preprocess_whole = Pipeline([("imputer", SimpleImputer(missing_values = -200, strategy = "mean")),
                             ("scaler", StandardScaler())])
whole_XTrain = preprocess_whole.fit_transform(whole_training_attribute_unproc.copy())
whole_yTrain = np.array(whole_training_target.copy())


# apply "preprocess_whole" pipeline to test data 
XTest_rf = preprocess_whole.transform(data_test_attribute_unproc.copy())
yTest_rf = np.array(data_test_target.copy())


# fit a random forest regression model to the whole training data and the best parameters 
regressor = RandomForestRegressor(n_estimators = grid_regressor.best_params_["n_estimators"],
                                 max_samples = grid_regressor.best_params_["max_samples"],
                                 max_features = grid_regressor.best_params_["max_features"])
regressor.fit(whole_XTrain, whole_yTrain)
yPred_rf = regressor.predict(XTest_rf)

from sklearn.metrics import mean_squared_error
error_rf = np.sqrt(mean_squared_error(yTest_rf, yPred_rf))
print("the RMSE over the test data is : {}".format(error_rf))

the RMSE over the test data is : 0.41815729923034195
